In [1]:
%load_ext autoreload
%autoreload 2
import os.path as op
import tempfile

import matplotlib.pyplot as plt


import hnn_core
from hnn_core import simulate_dipole, jones_2009_model
from hnn_core.viz import plot_dipole


import sys
sys.path.append('../code/')


import numpy as np
import torch

# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

# sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi

from summary_features.calculate_summary_features import calculate_summary_statistics_alternative as extract_sumstats



In [2]:
## defining neuronal network model

from utils.simulation_wrapper import event_seed, set_network_default, simulation_wrapper,simulation_wrapper_obs

sim_wrapper = simulation_wrapper_obs

net = set_network_default()

In [3]:
net._params

{
    "L2Basket_Gauss_A_weight": 0.0,
    "L2Basket_Gauss_mu": 2000.0,
    "L2Basket_Gauss_sigma": 3.6,
    "L2Basket_Pois_A_weight_ampa": 0.0,
    "L2Basket_Pois_A_weight_nmda": 0.0,
    "L2Basket_Pois_lamtha": 0.0,
    "L2Pyr_Gauss_A_weight": 0.0,
    "L2Pyr_Gauss_mu": 2000.0,
    "L2Pyr_Gauss_sigma": 3.6,
    "L2Pyr_Pois_A_weight_ampa": 0.0,
    "L2Pyr_Pois_A_weight_nmda": 0.0,
    "L2Pyr_Pois_lamtha": 0.0,
    "L2Pyr_ampa_e": 0.0,
    "L2Pyr_ampa_tau1": 0.5,
    "L2Pyr_ampa_tau2": 5.0,
    "L2Pyr_apical1_L": 306.0,
    "L2Pyr_apical1_diam": 4.08,
    "L2Pyr_apicaloblique_L": 340.0,
    "L2Pyr_apicaloblique_diam": 3.91,
    "L2Pyr_apicaltrunk_L": 59.5,
    "L2Pyr_apicaltrunk_diam": 4.25,
    "L2Pyr_apicaltuft_L": 238.0,
    "L2Pyr_apicaltuft_diam": 3.4,
    "L2Pyr_basal1_L": 85.0,
    "L2Pyr_basal1_diam": 4.25,
    "L2Pyr_basal2_L": 255.0,
    "L2Pyr_basal2_diam": 2.72,
    "L2Pyr_basal3_L": 255.0,
    "L2Pyr_basal3_diam": 2.72,
    "L2Pyr_dend_Ra": 200.0,
    "L2Pyr_dend_cm": 0.619

In [4]:
window_len = 30
prior_min = [7.9, 43.8]   # 't_evdist_1', 'sigma_t_evdist_1', 't_evprox_2', 'sigma_t_evprox_2'

prior_max = [30, 79.9]  

prior = utils.torchutils.BoxUniform(low=prior_min, 
                                    high=prior_max)

number_simulations = 8
density_estimator = 'nsf'

In [5]:

assert (prior.event_shape==torch.Size([2]))

In [6]:
from utils import inference
from utils.simulation_wrapper import event_seed, simulation_wrapper


## define number of summary statistics

sum_stats_number = 6
number_stats = 6

#posterior, theta, x, x_without = inference.run_sim_inference(prior, 
#                                                             sim_wrapper,
#                                                             calc_sum_stats=extract_sumstats,
#                                                             sum_stats_number = sum_stats_number,
#                                                             num_simulations=number_simulations, 
#                                                             density_estimator=density_estimator, 
#                                                             num_workers=8)

theta, x_without = inference.run_sim_theta_x(
            prior, 
            sim_wrapper,
            num_simulations = number_simulations,
            num_workers = 8
        )

2
params size torch.Size([2, 2])
2 params are investigated
joblib will run over 1 jobs
2
params size torch.Size([2])
2 params are investigated
joblib will run over 1 jobs
Loading custom mechanism files from /home/ubuntu/miniconda3/envs/sbi_env/lib/python3.7/site-packages/hnn_core/mod/x86_64/.libs/libnrnmech.so
Building the NEURON model
[Done]
running trial 1 on 1 cores
Simulation time: 0.03 ms...
Simulation time: 10.0 ms...
Simulation time: 20.0 ms...
Simulation time: 30.0 ms...
Simulation time: 40.0 ms...
Simulation time: 50.0 ms...
Simulation time: 60.0 ms...
Simulation time: 70.0 ms...
Simulation time: 80.0 ms...
Simulation time: 90.0 ms...
Simulation time: 100.0 ms...
Simulation time: 110.0 ms...
Simulation time: 120.0 ms...
Simulation time: 130.0 ms...
Simulation time: 140.0 ms...
Simulation time: 150.0 ms...
Simulation time: 160.0 ms...


Running 8 simulations in 8 batches.:   0%|          | 0/8 [00:00<?, ?it/s]

In [17]:
x = extract_sumstats(x_without, number_stats)

window_len, scaling_factor = 30, 3000

In [39]:
x.shape

torch.Size([100, 100])

In [37]:
from sbi.inference import SNPE_C

inf = SNPE_C(prior, density_estimator="nsf")

inf = inf.append_simulations(theta, x)


density_estimator = inf.train()

posterior = inf.build_posterior(density_estimator)

AssertionError: Number of parameter sets (=8 must match the number of simulation outputs (=100)

In [27]:
x.shape

torch.Size([100, 100])

In [31]:
from data_load_writer import write_to_file

import os
experiment_name = 'alternative_sum_stats_500sim_2params'
true_params = [26.6, 63.53]

#os.mkdir('results')


num_samples = 100
file_writer = write_to_file.WriteToFile(
    experiment=experiment_name,
    num_sim=number_simulations,
    true_params=true_params,
    density_estimator='nsf',
    num_params=2,
    num_samples=num_samples,
    )

os.mkdir('results/{}/step1'.format(experiment_name))


file_writer.save_posterior(posterior)
file_writer.save_observations(x, name='step1')

file_writer.save_obs_without(x_without, name='step1')
file_writer.save_thetas(theta, name='step1')


In [32]:
true_params = torch.tensor([[26.6, 63.53]])
obs_real = inference.run_only_sim(true_params, sim_wrapper)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
2
params size torch.Size([2])
2 params are investigated
joblib will run over 1 jobs
Building the NEURON model
2
params size torch.Size([2])
2 params are investigated
joblib will run over 1 jobs
Loading custom mechanism files from /home/ubuntu/miniconda3/lib/python3.9/site-packages/hnn_core/mod/x86_64/.libs/libnrnmech.so
Building the NEURON model
[Done]
running trial 1 on 1 cores
Simulation time: 0.03 ms...
Simulation time: 10.0 ms...
Simulation time: 20.0 ms...
Simulation time: 30.0 ms...
Simulation time: 40.0 ms...
Simulation time: 50.0 ms...
Simulation time: 60.0 ms...
Simulation time: 70.0 ms...
Simulation time: 80.0 ms...
Simulation time: 90.0 ms...
Simulation time: 100.0 ms...
Simulation time: 110.0 ms...
Simulation time: 120.0 ms...
Simulation time: 130.0 ms...
Simulation time: 140.0 ms...
Simulation time: 150.0 ms...
Simulation time: 160.0 ms...
2
params size torch.Size([2])
2 params are investigate

In [36]:
print(obs_real)
obs_real_stats = extract_sumstats(obs_real[0], number_stats)

print(obs_real_stats.shape)


samples = posterior.sample((num_samples,), x=obs_real_stats)


samples_prior = []


for i in range(num_samples):
    sample = prior.sample()
    samples_prior.append(sample)

[tensor([-0.5976, -0.5984, -0.5992,  ...,  7.6487,  7.6247,  7.6007],
       dtype=torch.float64)]
torch.Size([100])


Drawing 100 posterior samples:   0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (100) must match the size of tensor b (6801) at non-singleton dimension 1

In [ ]:
from joblib import Parallel, delayed

num_workers = 8
#x_lin = torch.arange(0,10,0.01, dtype= torch.float32)
s_x = inference.run_only_sim(samples, sim_wrapper, num_workers)

In [ ]:
s_x_prior = inference.run_only_sim(samples_prior, sim_wrapper, num_workers)

In [ ]:
s_x_stat = extract_sumstats(s_x, number_stats)

s_x_prior_stat = extract_sumstats(s_x_prior, number_stats)


print(s_x_stats.shape)


In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_title("Simulating from posterior")
for s in s_x:
    plt.plot(s, alpha=0.1, color='blue')
    #plt.ylim(-30,30)
    plt.xlim(0, 7000)
plt.plot(obs_real[0], label='Ground truth', color='red')
#plt.legend()
    
    
    
fig, ax = plt.subplots(1, 1)
ax.set_title("Simulating from prior")
for x_w in s_x_prior:
    plt.plot(x_w, alpha=0.1, color='blue')

In [ ]:
import matplotlib.gridspec as gridspec



fig = plt.figure(figsize=(10,40), tight_layout=True)

gs = gridspec.GridSpec(nrows=x.size(dim=1), ncols=1)
print(len(s_x_stat[0]))

sum_stats_names = torch.arange(1, len(s_x_stat[0])+1, 1)

print(sum_stats_names)



#fig.suptitle('Summary stats histogram from posterior predictions.', y=0.2, fontsize=16)


for i in range(len(sum_stats_names)):

    globals()['ax%s' % i] = fig.add_subplot(gs[i])

    globals()['sum_stats%s' % i] = []
    globals()['x%s' % i] = []

    for j in range(len(s_x)):
        globals()['sum_stats%s' % i].append(s_x_stat[j][i])
        globals()['x%s' % i].append(s_x_prior_stat[j][i])



    globals()['ax%s' % i].hist(globals()['sum_stats%s' % i],  density=False, facecolor='g', alpha=0.75, histtype='barstacked', label='from posterior')
    globals()['ax%s' % i].hist(globals()['x%s' % i],  density=False, facecolor='b', alpha=0.5, histtype='barstacked', label='simulated')
    
  
    globals()['ax%s' % i].set_title('Histogram of summary stat "{}" '.format(sum_stats_names[i]), pad=20)
    #ax0.set(ylim=(-500, 7000))

    globals()['ax%s' % i].axvline(obs_real_stats[0][i], color='red', label='ground truth')
    globals()['ax%s' % i].legend(loc='upper right')





plt.savefig('Summary_stats_post_prior')

In [ ]:
true_params = torch.Tensor([26.6, 63.53])




fig, axes = analysis.pairplot(samples,
                           #limits=[[.5,80], [1e-4,15.]],
                           #ticks=[[.5,80], [1e-4,15.]],
                           figsize=(5,5),
                           points=true_params,
                           points_offdiag={'markersize': 6},
                           points_colors='r');